## Zigbang 원룸 매물 데이터 수집

In [1]:
import requests
import pandas as pd

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

### 1\. 동이름으로 위도 경도 구하기

In [3]:
addr = '망원동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'

response = requests.get(url)
response

<Response [200]>

In [8]:
response.text

'{"success":true,"code":"200","items":[{"id":4002,"type":"address","name":"망원동","hint":"","description":"서울특별시 마포구 망원동","lat":37.556785583496094,"lng":126.9013442993164,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":3,"local1":"서울시","local2":"마포구","local3":"망원동","web_level":15,"web_lat":37.556785583496094,"web_lng":126.9013442993164,"app_level":15,"app_lat":37.556785583496094,"app_lng":126.9013442993164,"법정동코드":"1144012300"},"zoom_level":{"google":15,"daum":4},"zoom_level_v2":{"app":5,"web":4}}],"next":null,"limit":0}'

In [12]:
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(37.556785583496094, 126.9013442993164)

### 2\. 위도 경도로 geohash 알아내기

In [14]:
# install geohash2
!pip install geohash2

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15554 sha256=a96755d1c892b3bd38fde0368cd2ed4fce525378f651afeb20f99ee65c2bb30a
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\f6\7c\c4\1b3c6fea0ebc53bf730dc86bbee7a713d501455dfb4c1f0623
Successfully built geohash2


In [15]:
import geohash2

In [17]:
geohash = geohash2.encode(lat, lng, precision=5) # precision: 영역 -> 숫자가 커질 수록 영역의 규모가 작아짐
geohash

'wydjx'

### 3\. geohash로 매물 아이디 가져오기

In [25]:
url = f'https://apis.zigbang.com/v2/items/oneroom?\
geohash={geohash}&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세\
&domain=zigbang&checkAnyItemWithoutFilter=true'

response = requests.get(url)
response

<Response [200]>

In [26]:
# response.text[:500]

items = response.json()['items']
items[:3]

[{'lat': 37.52957191936743,
  'lng': 126.900435212516,
  'itemId': 39817840,
  'itemBmType': 'ZIGBANG'},
 {'lat': 37.52955993739013,
  'lng': 126.89979844625006,
  'itemId': 40042434,
  'itemBmType': 'ZIGBANG'},
 {'lat': 37.52907304257156,
  'lng': 126.90042820847293,
  'itemId': 40058332,
  'itemBmType': 'ZIGBANG'}]

In [27]:
items = [item['itemId'] for item in items]
len(items), items[:5]

(605, [39817840, 40042434, 40058332, 39791961, 40096518])

### 4\. 매물 아이디로 매물 정보 가져오기

In [29]:
url = 'https://apis.zigbang.com/v2/items/list'
params = {
    'domain': 'zigbang',
    'item_ids': items
}

response = requests.post(url, params) # params에 한글이 없으므로 json.dumps로 인코딩할 필요 없음
response

<Response [200]>

In [37]:
pd.options.display.max_columns=40 # 원하는 갯수만큼 컬럼 출력

In [43]:
data = response.json()['items']
columns = ['item_id','sales_title', 'deposit', 'rent', 'size_m2', 'title', 'address1']
df = pd.DataFrame(data)[columns]

# 망원동 옆 동인 중동의 데이터도 가져오므로 망원동만 가져오도록 필터링
df = df[df['address1'].str.contains('망원동')].reset_index(drop=True)
df.tail(2)

,item_id,sales_title,deposit,rent,size_m2,title,address1
61,40009046,월세,500,50,13.22,신축급풀옵션마포구청역5분깔끔한방,서울시 마포구 망원동
62,40089526,월세,200,63,16.53,신축급 깔끔원룸,서울시 마포구 망원동


In [40]:
df.columns

Index(['item_id', 'section_type', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'status', 'tags', 'service_type',
       'random_location', 'manage_cost', 'reg_date', 'is_new', 'addressOrigin',
       'action', 'contract', 'address', 'is_zzim', 'address1', 'address2',
       'address3', 'item_bm_type', 'zikim'],
      dtype='object')